# Fetch Rewards Senior Data Analyst Take Home Assessment

### Data Exploration

In [1]:
# Importing Necessary Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
# Creating dataframes for the csv files provided

users = pd.read_csv('data/USER_TAKEHOME.csv')
txns = pd.read_csv('data/TRANSACTION_TAKEHOME.csv')
products = pd.read_csv('data/PRODUCTS_TAKEHOME.csv')

##### Users

In [3]:
# Running users.head() to get an idea of what the data looks like

users.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female


The users dataframe is pretty straightforward and easy to understand. It consists of basic information about users including their gender and language. I can assume that en is english, and also assume that es-419 is espagnol, but I am curious about the meaning of 419. Next, I'm going to check the datatypes included in the df and change if needed. 

In [4]:
# Getting datatypes in the dataframe

print(users.dtypes)

ID              object
CREATED_DATE    object
BIRTH_DATE      object
STATE           object
LANGUAGE        object
GENDER          object
dtype: object


Since the object datatype can be less efficient, I'm going to change the datatypes to strings or datetimes wherever possible. I'll start with the date/time columns in this df, which seems to be the only ones that need to be changed.

In [5]:
# First stripping Z from the end of both columns
# The Z in this column denotes the time zone, which is UTC

users['CREATED_DATE'] = users['CREATED_DATE'].str.replace('Z', '')
users['BIRTH_DATE'] = users['BIRTH_DATE'].str.replace('Z', '')

In [6]:
# creating a function for datetime in case I need to cast other columns throughout the data exploration process

def cast_datetime(df, cols):
    # Ensure columns exist
    cols = [col for col in cols if col in df.columns]

    for col in cols:
        df[col] = df[col].map(lambda x: pd.NaT if pd.isna(x) else 
                              (x.get('$date') if isinstance(x, dict) else x))
        df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)

    return df

# Convert date columns in one step
date_cols = ['CREATED_DATE', 'BIRTH_DATE']
users = cast_datetime(users, date_cols)

In [7]:
# Creating a function to return the amount of null values in the df which I can use again llater

def null_summary(df):
    total_rows = len(df)
    null_counts = df.isnull().sum()
    percent_null = round((null_counts / total_rows) * 100, 2)

    summary_df = pd.DataFrame({
        'Column': null_counts.index,
        'Total Nulls': null_counts.values,
        'Total Rows': total_rows,
        'Percent Null': percent_null.values
    })

    return summary_df.sort_values(by='Percent Null', ascending=False)

# Call the function
null_summary(users)

,Column,Total Nulls,Total Rows,Percent Null
4,LANGUAGE,30508,100000,30.51
5,GENDER,5892,100000,5.89
3,STATE,4812,100000,4.81
2,BIRTH_DATE,3675,100000,3.68
0,ID,0,100000,0.00
1,CREATED_DATE,0,100000,0.00


The amount of null values for language in this dataset is concerning, because it could become problematic if the default language used isn't properly communicating with the user. 

##### Transactions

In [8]:
# Getting an idea of what the txns df looks like 

txns.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


Right away, I notice that I'm going to need to change a few datatypes and replace some values. However, the df is very easy to understand. In this df, `RECEIPT_ID` is the primary key, and `USER_ID` is the foreign key which can be used to join with the users df. Barcode looks like it is a number and *should* be a number, but `FINAL_QUANTITY` will require some work. The zero's will need to be replaced with 0.0, and then changed to a float. This will need to be a float because it's possible that a user purchases something that goes by weight instead of quantity. This would need further clarification. 

In [9]:
print(txns.dtypes)

RECEIPT_ID         object
PURCHASE_DATE      object
SCAN_DATE          object
STORE_NAME         object
USER_ID            object
BARCODE           float64
FINAL_QUANTITY     object
FINAL_SALE         object
dtype: object


As expected, most of these columns are objects and will need to be changed. I will also be changing the barcode column to an integer instead of a float. 

In [10]:
date_cols = ['PURCHASE_DATE', 'SCAN_DATE']

txns = cast_datetime(txns, date_cols)

In [11]:
# Calling null_summary function to check on the null values in the df

null_summary(txns)

,Column,Total Nulls,Total Rows,Percent Null
5,BARCODE,5762,50000,11.52
0,RECEIPT_ID,0,50000,0.00
1,PURCHASE_DATE,0,50000,0.00
2,SCAN_DATE,0,50000,0.00
3,STORE_NAME,0,50000,0.00
4,USER_ID,0,50000,0.00
6,FINAL_QUANTITY,0,50000,0.00
7,FINAL_SALE,0,50000,0.00


In [12]:
# Changing the zero's to 0.0 so the column can be converted to a float

txns['FINAL_QUANTITY'] = txns['FINAL_QUANTITY'].str.replace('zero', '0.0').astype(float)

txns['FINAL_SALE'] = pd.to_numeric(txns['FINAL_SALE'], errors='coerce').fillna(0).astype(float)


In [13]:
# Verifying that the datatypes changed with the code that has been run

print(txns.dtypes)

RECEIPT_ID                     object
PURCHASE_DATE     datetime64[ns, UTC]
SCAN_DATE         datetime64[ns, UTC]
STORE_NAME                     object
USER_ID                        object
BARCODE                       float64
FINAL_QUANTITY                float64
FINAL_SALE                    float64
dtype: object


### Products

In [14]:
# Looking at products df

products.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11


The data in this dataframe is also very self-explanatory. `CATEGORY_1` is the highest level of categories, and `CATEGORY_4` is the lowest subcategory. I expect all of the fields in this df to be an object dtype, except for barcode. 

In [15]:
print(products.dtypes)

CATEGORY_1       object
CATEGORY_2       object
CATEGORY_3       object
CATEGORY_4       object
MANUFACTURER     object
BRAND            object
BARCODE         float64
dtype: object


In [16]:
null_summary(products)

,Column,Total Nulls,Total Rows,Percent Null
3,CATEGORY_4,778093,845552,92.02
4,MANUFACTURER,226474,845552,26.78
5,BRAND,226472,845552,26.78
2,CATEGORY_3,60566,845552,7.16
6,BARCODE,4025,845552,0.48
1,CATEGORY_2,1424,845552,0.17
0,CATEGORY_1,111,845552,0.01


The null summary for this df is very helpful to understand. It's crucial that at the very least, `CATEGORY_1` is almost always populated with data. However, `BARCODE` is null 48% of the time, and `BRAND` is null almost 27% of the time. This points to a data quality issue. A lot of important information and insight can be gathered from this data, so I would be curious to know why the brands may be null.

### Exporting to CSV's for SQL Querying

In [18]:
users.to_csv('/Users/deannagould/Documents/Bucks/github/fetch-rewards-sr_data_analyst/csv_files/users.csv', index=False)
txns.to_csv('/Users/deannagould/Documents/Bucks/github/fetch-rewards-sr_data_analyst/csv_files/txns.csv', index=False)
products.to_csv('/Users/deannagould/Documents/Bucks/github/fetch-rewards-sr_data_analyst/csv_files/products.csv', index=False)